In [1]:
#imports
import pandas as pd 
import plotly as pl
import plotly.express as px

In [ ]:
# import dataset

DATA = pd.read_csv("mammographic_masses_data.csv")


In [ ]:
# plot the data

x = DATA['Age']
y = DATA['Density']

fig1 = px.scatter(x=x, y=y, labels={'x': 'Age', 'y': 'Density'}, title='Age plotted against Denisty')

fig1.show()